@author: Octavio Gutiérrez de Código Máquina

URL del canal: https://www.youtube.com/CodigoMaquina

URL del video: https://youtu.be/PFn31_hzQ2Y 

<img src="imagenes/basketball.jpg" width="800" align="center">


# Contexto y Datos

In [1]:
import pandas as pd

puntos_partido = pd.Series(["alto", "bajo", "alto", "alto", "alto",
                            "alto", "bajo", "alto", "alto", "bajo"])

minutos_partido = pd.Series(["alto", "alto", "bajo", "bajo", "bajo",
                             "alto", "bajo", "bajo", "bajo", "alto"])

rebotes_partido = pd.Series(["alto", "bajo", "bajo", "alto", "bajo",
                             "alto", "bajo", "alto", "bajo", "alto"])

asistencias_partido = pd.Series(["bajo", "bajo", "bajo", "bajo", "bajo",
                                 "bajo", "bajo", "bajo", "bajo", "bajo"])

# 1: Veterano (carrera de cinco años o más)
clase = pd.Series([1, 0, 0, 1, 0, 1, 0, 1, 0, 1])

datos = pd.DataFrame({"puntos": puntos_partido,
                      "minutos": minutos_partido,
                      "asistencias": asistencias_partido,
                      "rebotes": rebotes_partido,                      
                      "clase": clase})

<BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR>

# Interpretación de la Impureza GINI

In [2]:
def impureza_gini(caracteristica, clase, datos):
    """str, str, DataFrame -> float"""    
    atributo_clase = datos.groupby([caracteristica, clase])[clase].count()
    atributo = datos.groupby([caracteristica])[clase].count()
    procesados = pd.merge(atributo_clase, atributo, on=[caracteristica], 
                          suffixes=('_individual', '_total')) 
    procesados["combinacion"] = (procesados[clase+"_individual"]/
                                 procesados[clase+"_total"])**2
    gini_combinacion = 1 - procesados.groupby([caracteristica, clase+"_total"])["combinacion"].sum()
    gini_pesado = (gini_combinacion * atributo) / atributo.sum() 
    return gini_pesado.sum()


print("puntos -> impureza: %0.4f" % impureza_gini("puntos", "clase", datos))
print("minutos -> impureza: %0.4f" % impureza_gini("minutos", "clase", datos))
print("asistencias -> impureza: %0.4f" % impureza_gini("asistencias", "clase", datos))
print("rebotes -> impureza: %0.4f" % impureza_gini("rebotes", "clase", datos)) 

datos

puntos -> impureza: 0.4762
minutos -> impureza: 0.4167
asistencias -> impureza: 0.5000
rebotes -> impureza: 0.0000


,puntos,minutos,asistencias,rebotes,clase
0,alto,alto,bajo,alto,1
1,bajo,alto,bajo,bajo,0
2,alto,bajo,bajo,bajo,0
3,alto,bajo,bajo,alto,1
4,alto,bajo,bajo,bajo,0
5,alto,alto,bajo,alto,1
6,bajo,bajo,bajo,bajo,0
7,alto,bajo,bajo,alto,1
8,alto,bajo,bajo,bajo,0
9,bajo,alto,bajo,alto,1


<BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR>

# Impureza del atributo 'puntos'

In [3]:
# Cuántos jugadores por categoría de puntos y por clase (novato o veterano)
puntos_y_clase = datos.groupby(["puntos", "clase"])["clase"].count()

print("\n\nJUGADORES POR PUNTOS Y CLASE\n\n", puntos_y_clase)

# Cuántos jugadores por categoría de puntos 
puntos = datos.groupby(["puntos"])["clase"].count()

print("\n\nJUGADORES POR PUNTOS\n\n", puntos)

# Unir ambas series de datos para procesamiento posterior 
jugadores = pd.merge(
    puntos_y_clase,
    puntos,
    on=["puntos"], 
    suffixes=('_indivual', '_total')) 

print("\n\nUNION\n\n", jugadores)




JUGADORES POR PUNTOS Y CLASE

 puntos  clase
alto    0        3
        1        4
bajo    0        2
        1        1
Name: clase, dtype: int64


JUGADORES POR PUNTOS

 puntos
alto    7
bajo    3
Name: clase, dtype: int64


UNION

         clase_indivual  clase_total
puntos                             
alto                 3            7
alto                 4            7
bajo                 2            3
bajo                 1            3


# Cálculo de la Impureza GINI para 'puntos'
# 

<img src="imagenes/formula_gini.png" width="325" align="left">

In [4]:
# Probabilidad para cada categoría de puntos con respecto a la clase
jugadores["combinaciones"] = (jugadores["clase_indivual"]/jugadores["clase_total"])**2
print(jugadores)

# Impureza gini para cada combinación
gini_por_combinacion = 1 - jugadores.groupby(["puntos", "clase_total"])["combinaciones"].sum()
print("\n\n",gini_por_combinacion)

# Impureza gini para cada combinación con pesos
gini_con_peso_por_combinacion = (gini_por_combinacion * puntos) / puntos.sum() 
print("\n\n", gini_con_peso_por_combinacion)
print("\n", gini_con_peso_por_combinacion.sum())



        clase_indivual  clase_total  combinaciones
puntos                                            
alto                 3            7       0.183673
alto                 4            7       0.326531
bajo                 2            3       0.444444
bajo                 1            3       0.111111


 puntos  clase_total
alto    7              0.489796
bajo    3              0.444444
Name: combinaciones, dtype: float64


 puntos  clase_total
alto    7              0.342857
bajo    3              0.133333
dtype: float64

 0.4761904761904763


<BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR><BR>